# San Francisco Crime Classification

## 📌 데이터셋 소개

**2013년 1월부터 2015년 5월까지 샌프란시스코에서 발생한 범죄 데이터**

- Dates: 범죄 발생 시간
- Category: 범죄 카테고리
- Descript: 해당 사건에 대한 설명
- DayOfWeek: 사건이 발생한 요일
- PdDistrict: 관할 경찰서
- Resolution: 해결 방법
- Address: 범죄 장소
- X: 위도
- Y: 경도

### 최종 분석 목표 : 샌프란시스코 범죄 데이터 분석을 이용하여 거리 범죄를 낮추기 위한 방안 마련
>
> - 특정 지역에서 범죄가 발생할 경우... 어떤 카테고리에 속한 범죄일지? 
시간 장소 등을 이용해서 카테고리를 찾는거 -> 분류 문제

> - 특정 장소 특정 지역 순찰조 편성 : 어느 관할서가, 어떤 요일/시간에, 어떤 범죄를 위주로 예방하기 위해 효율적으로 순찰할 것인지
지도 출력해서 관할서 위치도 찾아서 표시하고
요일/ 시간 분석해서 어떤 범죄가 일어나는지 표시하고
표시된거 바탕으로 순찰조 위치 표시 시간대 인원수 

## 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import os

## 1차 데이터 전처리

In [2]:
# 데이터 불러오기
path = os.getenv('HOME') + '/aiffel/project_code/Datatone'
data = pd.read_csv(path + "/train.csv")

# 데이터 갯수 출력
print(data.shape[0])

878049


In [3]:
# 데이터 정보 출력하기
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 878049 entries, 0 to 878048
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Dates       878049 non-null  object 
 1   Category    878049 non-null  object 
 2   Descript    878049 non-null  object 
 3   DayOfWeek   878049 non-null  object 
 4   PdDistrict  878049 non-null  object 
 5   Resolution  878049 non-null  object 
 6   Address     878049 non-null  object 
 7   X           878049 non-null  float64
 8   Y           878049 non-null  float64
dtypes: float64(2), object(7)
memory usage: 60.3+ MB


In [4]:
# 결측치 처리하기
data.isnull().sum()

Dates         0
Category      0
Descript      0
DayOfWeek     0
PdDistrict    0
Resolution    0
Address       0
X             0
Y             0
dtype: int64

#### 결측치 없음

In [5]:
# 중복값 확인하기
print("중복값 개수 : ", data.duplicated().sum())

중복값 개수 :  2323


In [6]:
# 중복값 제거하기
data.drop_duplicates(inplace = True)

# 중복 처리 후 데이터 갯수 확인
print("남은 데이터 수", data.shape[0])

남은 데이터 수 875726


In [7]:
# string -> datetime 형태로 변환
data['Dates'] = pd.to_datetime(data['Dates'])

# 'Dates' 열 년, 월, 일, 시로 분리
data['Year'] = data['Dates'].dt.year
data['Month'] = data['Dates'].dt.month
data['Day'] = data['Dates'].dt.day
data['Hour'] = data['Dates'].dt.hour
data['Minute'] = data['Dates'].dt.minute
data['Second'] = data['Dates'].dt.second

In [8]:
# 'Dates' 열 변환 확인
data[['Year','Month','Day','Hour','Minute','Second']]

,Year,Month,Day,Hour,Minute,Second
0,2015,5,13,23,53,0
1,2015,5,13,23,53,0
2,2015,5,13,23,33,0
3,2015,5,13,23,30,0
4,2015,5,13,23,30,0
...,...,...,...,...,...,...
878044,2003,1,6,0,15,0
878045,2003,1,6,0,1,0
878046,2003,1,6,0,1,0
878047,2003,1,6,0,1,0


### 데이터 전처리 정리
>
> - 결측치 없음
> - 중복값 2323개 제거
> - 'Dates'열 시간 단위 별로 분리

## 데이터 EDA + 2차 데이터 전처리

In [9]:
# 범죄 카테고리 확인
data['Category'].value_counts()

LARCENY/THEFT                  174320
OTHER OFFENSES                 125960
NON-CRIMINAL                    91915
ASSAULT                         76815
DRUG/NARCOTIC                   53919
VEHICLE THEFT                   53706
VANDALISM                       44581
WARRANTS                        42145
BURGLARY                        36600
SUSPICIOUS OCC                  31394
MISSING PERSON                  25669
ROBBERY                         22988
FRAUD                           16637
FORGERY/COUNTERFEITING          10592
SECONDARY CODES                  9979
WEAPON LAWS                      8550
PROSTITUTION                     7446
TRESPASS                         7318
STOLEN PROPERTY                  4537
SEX OFFENSES FORCIBLE            4380
DISORDERLY CONDUCT               4313
DRUNKENNESS                      4277
RECOVERED VEHICLE                3132
KIDNAPPING                       2340
DRIVING UNDER THE INFLUENCE      2268
LIQUOR LAWS                      1899
RUNAWAY     

## 범죄 카테고리 번역
> - LARCENY/THEFT - 절도(경범죄)
> - OTHER OFFENSES - 기타 범죄
> - NON-CRIMINAL - 비범죄
> - ASSAULT - 폭행
> - DRUG/NARCOTIC - 마약/마약류
> - VEHICLE THEFT - 차량 절도
> - VANDALISM - 기물 파손
> - WARRANTS - 구속
> - BURGLARY - 절도(중범죄)
> - SUSPICIOUS OCC - 의심스러운 사건
> - MISSING PERSON - 실종
> - ROBBERY - 강도
> - FRAUD - 사기
> - FORGERY/COUNTERFEITING - 위조/위조품
> - SECONDARY CODES - 2차적 배경들이 있는 사건
> - WEAPON LAWS - 총기 사고
> - PROSTITUTION - 매춘
> - TRESPASS - 무단 침입
> - STOLEN PROPERTY - 도난당한 재산
> - SEX OFFENSES FORCIBLE - 강제 성범죄
> - DISORDERLY CONDUCT - 문란행위
> - DRUNKENNESS - 음주
> - RECOVERED VEHICLE - 차량 탈취
> - KIDNAPPING - 납치
> - DRIVING UNDER THE INFLUENCE - 음주운전
> - LIQUOR LAWS - 주류 관련 법규 위반
> - RUNAWAY - 가출
> - ARSON - 방화
> - LOITERING - 의심스로운 배회 및 방황
> - EMBEZZLEMENT - 횡령
> - SUICIDE - 자살
> - FAMILY OFFENSES - 가족 관련 범죄
> - BAD CHECKS - 부도 수표
> - BRIBERY - 뇌물
> - EXTORTION - 갈취
> - SEX OFFENSES NON FORCIBLE - 강제가 아닌 성범죄
> - GAMBLING - 도박
> - PORNOGRAPHY/OBSCENE MAT - 음란행위, 외설행위
> - TREA - 무단침입

### 주제와 벗어나는 범죄 리스트
- NON-CRIMINAL, WARRANTS, SUSPICIOUS OCC, FRAUD, FORGERY/COUNTERFEITING, SECONDARY CODES, PROSTITUTION, DRIVING UNDER THE INFLUENCE, LIQUOR LAWS, RUNAWAY, EMBEZZLEMENT, SUICIDE, BAD CHECKS, BRIBERY, GAMBLING

In [13]:
# 거리 범죄만 남기기
drop_crimes = ['NON-CRIMINAL', 'WARRANTS', 'SUSPICIOUS OCC', 'FRAUD', 'FORGERY/COUNTERFEITING', 'SECONDARY CODES', 'PROSTITUTION', 'DRIVING UNDER THE INFLUENCE', 'LIQUOR LAWS', 'RUNAWAY', 'EMBEZZLEMENT', 'SUICIDE', 'BAD CHECKS', 'BRIBERY', 'GAMBLING']

city_crimes = data[~data['Category'].isin(drop_crimes)]
city_crimes['Category'].value_counts()

LARCENY/THEFT                174320
OTHER OFFENSES               125960
ASSAULT                       76815
DRUG/NARCOTIC                 53919
VEHICLE THEFT                 53706
VANDALISM                     44581
BURGLARY                      36600
MISSING PERSON                25669
ROBBERY                       22988
WEAPON LAWS                    8550
TRESPASS                       7318
STOLEN PROPERTY                4537
SEX OFFENSES FORCIBLE          4380
DISORDERLY CONDUCT             4313
DRUNKENNESS                    4277
RECOVERED VEHICLE              3132
KIDNAPPING                     2340
ARSON                          1512
LOITERING                      1207
FAMILY OFFENSES                 488
EXTORTION                       256
SEX OFFENSES NON FORCIBLE       148
PORNOGRAPHY/OBSCENE MAT          22
TREA                              6
Name: Category, dtype: int64

In [14]:
print("남은 데이터 수", city_crimes.shape[0])

남은 데이터 수 657044


## 1번

## 2번

## 3번

## 4번

## 최종 결론